In [1]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from tqdm import tqdm
import pandas as pd
from rank_bm25 import BM25Okapi 
import ir_datasets

## Collecting Data

In [2]:
dataset = ir_datasets.load("neuclir/1/multi/trec-2023")

In [ ]:
def random_sample(iterator, sample_size, total_count):
    sample_probability = sample_size / total_count
    sampled_data = []

    for item in iterator:
        if random.random() < sample_probability:
            sampled_data.append(item)
        if len(sampled_data) >= sample_size:
            break

    return pd.DataFrame(sampled_data)

qrels_df = pd.DataFrame(dataset.qrels_iter())
documents_df = random_sample(dataset.docs_iter(), 2000000, dataset.docs_count())
queries_df = pd.DataFrame(dataset.queries_iter())

queries_df = pd.DataFrame(dataset.queries_iter())
documents_df = documents_df[documents_df["doc_id"].isin(qrels_doc_ids)]

In [4]:
documents_df = pd.read_csv('/Users/mjlee/Documents/Study/HW_2024/IR/Project/MLIR/data/sampled_doc.csv')
qrels_df = pd.DataFrame(dataset.qrels_iter())
queries_df = pd.DataFrame(dataset.queries_iter())
qrels_doc_ids = set(qrels_df["doc_id"])
qrels_df = pd.DataFrame(dataset.qrels_iter())
documents_df = documents_df[documents_df["doc_id"].isin(qrels_doc_ids)]

In [ ]:
documents_df.head()

,Unnamed: 0,doc_id,title,text,url,time,cc_file
0,163,14a0a786-c707-487c-af0a-16ef97e1d023,越捷航空連三年獲評為「最佳超低廉航空公司」 - 生活,AirlineRatings頒發越捷航空 2020「最佳超低廉航空公司」殊榮。圖：越捷提供\...,https://www.chinatimes.com/realtimenews/201911...,2019-11-27T15:44:07+08:00,crawl-data/CC-NEWS/2019/11/CC-NEWS-20191127083...
1,232,fc24998f-6ed8-43f5-93aa-f71244cd50af,精品包用酒精消毒小心壞掉！快來學4招正確、實用的「清潔技巧」,疫情期間只要一出門，回到家就要把隨身物品消毒一次，那麼大家都是如何幫心愛的精品包消毒呢？其實...,https://fashion.ettoday.net/news/2024734?from=...,2021-07-08T09:09:00+08:00,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708003...
2,250,b6c8165f-b19a-4d47-84b9-0c7c7e450cf6,一室枯山水 喝一杯手冲咖啡忘憂 - 20210708 - 副刊,手搖凍啡 入口柔順\n\n一般禪味的café均以茶品為主，不過店家主打咖啡，設有手冲咖啡菜單...,https://news.mingpao.com/pns/%E5%89%AF%E5%88%8...,NaN,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708003...
3,301,42dfc137-e3b8-4ea5-ba96-6914dc6afc1f,《電子零件》車用、儲能雙引擎 系統電6月營收月增5.97％ - 財經,系統電(5309)因車用及儲能產品需求強勁，6月合併營收為2.13億元，月成長5.97%，系...,https://www.chinatimes.com/realtimenews/202107...,2021-07-08T09:44:29+08:00,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708022...
4,306,e6887499-98df-4408-a6aa-c002a76cc801,富商遇死劫！小20歲嫩妻崩潰痛哭 遭警識破「全是演戲」,記者張寧倢／編譯\n\n印尼巴布亞省（Papua）於6月28日發生一起劫車殺人的嚴重社會案件...,https://www.ettoday.net/news/20210708/2025480....,2021-07-08T10:40:00+08:00,crawl-data/CC-NEWS/2021/07/CC-NEWS-20210708041...


In [ ]:
queries_df.head()

,query_id,title,description,narrative,fa_mt_title,fa_mt_description,fa_mt_narrative,ru_mt_title,ru_mt_description,ru_mt_narrative,zh_mt_title,zh_mt_description,zh_mt_narrative
0,200,Corruption Bribery Sports Federation Olympics,Are there cases of Institutional Corruption an...,We are Looking for articles that contain a cas...,المپیک فدراسیون ورزشی رشوه خواری فساد,آیا مواردی از فساد نهادی و رشوه خواری در ورزش ...,ما به دنبال مقالاتی هستیم که حاوی یک مورد فساد...,Олимпийские игры Федерации спортивных игр корр...,Существуют ли случаи институциональной коррупц...,"Мы ищем статьи, которые содержат случай финанс...",腐败贿赂体育联合会奥运会,体育中是否存在机构腐败和贿赂的案例？,我们正在寻找在机构层面上包含金融腐败或贿赂案件的文章。奥运会和体育联合会中的贿赂案件是相关的...
1,201,China investment in Iran,In what fields or industries China’s investmen...,We need articles that specify at least one fie...,سرمایه گذاری چین در ایران,در کدام زمینه ها یا صنایع سرمایه گذاری چین در ...,ما به مقالاتی نیاز داریم که حداقل یک زمینه سرم...,Китайские инвестиции в Иран,В каких областях или промышленности инвестиции...,"Нам нужны статьи, в которых указано хотя бы од...",中国对伊朗的投资,在哪些领域或行业中，中国对伊朗的投资是？,我们需要至少指定一个投资领域的文章，无论是皮带和道路倡议，石油，水坝等。货币的投资金额不提及...
2,202,"Emerging technologies, precision farming, smar...",What are some of the latest emerging technolog...,Find articles on emerging technologies in prec...,فن آوری های نوظهور ، کشاورزی دقیق ، کشاورزی هو...,برخی از جدیدترین فن آوری های نوظهور که صنعت کش...,مقالاتی در مورد فن آوری های نوظهور در کشاورزی ...,"Новые технологии, точное сельское хозяйство, у...",Каковы некоторые из последних новых технологий...,Найдите статьи о новых технологиях в области т...,新兴技术，精确农业，智能农业，农业,哪些最新的新兴技术正在改变农业行业？,查找有关精确农业和农业中新兴技术的文章，这些文章提供了这些技术如何用于提高农作物产量并提高农...
3,203,Ever Given's Stuck,Find information about the ship Ever Given bei...,Find articles on the event when the container ...,تا به حال گیر کرده است,اطلاعات مربوط به کشتی را که در کانال سوئز گیر ...,مقالاتی را در مورد این رویداد پیدا کنید که کشت...,Когда -либо дал застрял,"Найдите информацию о корабле, когда -либо заст...","Найдите статьи на мероприятии, когда контейнер...",曾经被卡住了,查找有关被困在苏伊士运河的船的信息,在2021年春季遇到的集装箱船被困在苏伊士运河中时，请查找有关事件的事件的文章。包括有关事件...
4,204,Penalty doping sports stop,What strategies are used to reduce/stop doping...,Any mention of the monetary penalties or exclu...,مجازات دوپینگ ورزش توقف,از چه استراتژی هایی برای کاهش/متوقف کردن دوپین...,هرگونه ذکر مجازات های پولی یا محرومیت تیم ها ی...,Спортивная остановка пенальти,Какие стратегии используются для уменьшения/ос...,Любое упоминание о денежных наказаниях или иск...,点球兴奋剂运动停止,哪些策略用于减少运动中的掺杂？,包括任何提及货币罚款或排除在团队中或夺回奖牌的罚款。接受掺杂的测试将不被视为预防措施。


## Preprocessing

In [5]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

documents_df['text'] = documents_df['text'].fillna('') 
documents_df = documents_df[documents_df['text'].str.strip() != '']

DetectorFactory.seed = 0

def safe_detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown" 

documents_df['language'] = documents_df['text'].apply(safe_detect_language)


documents_df['language'] = documents_df['language'].replace({'zh-tw': 'zh', 'zh-cn': 'zh', "ko": 'zh'})
documents_df = documents_df[documents_df['language'] != 'unknown']

print(documents_df['language'].value_counts())

documents_by_language = {
    lang: docs.reset_index(drop=True)
    for lang, docs in documents_df.groupby('language')
}

language
zh    5178
fa    5050
ru    4970
en      65
vi      27
no      11
es       5
pt       4
ca       3
fr       3
bg       3
cy       2
ar       2
sw       1
et       1
Name: count, dtype: int64


In [6]:
query_languages = ['fa', 'ru', 'zh']
queries_multilingual = {lang: queries_df[['query_id', f'{lang}_mt_title']].rename(columns={f'{lang}_mt_title': 'query_text'}) for lang in query_languages}

## Ranking Documents using BM25

In [7]:
bm25_results = {}

for query_lang, query_df in queries_multilingual.items():
    print(f"Processing BM25 for queries in {query_lang}...")

    # Select the corresponding document set
    if query_lang in documents_by_language:
        docs = documents_by_language[query_lang]
        doc_texts = docs['text'].tolist()
        doc_ids = docs['doc_id'].tolist()

        bm25 = BM25Okapi([doc.split() for doc in doc_texts])  # Tokenize documents

        # Process each query
        for _, row in tqdm(query_df.iterrows(), total=len(query_df), desc=f"Retrieving for {query_lang}"):
            query_id = row['query_id']
            query_text = row['query_text']
            
            # Tokenize query
            query_tokens = query_text.split()
            
            scores = bm25.get_scores(query_tokens)
            
            top_n = 10
            top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
            top_docs = [(doc_ids[i], scores[i]) for i in top_indices]

            # Store results
            bm25_results[(query_id, query_lang)] = top_docs

Processing BM25 for queries in fa...


Retrieving for fa: 100%|██████████| 76/76 [00:00<00:00, 123.47it/s]


Processing BM25 for queries in ru...


Retrieving for ru: 100%|██████████| 76/76 [00:00<00:00, 154.30it/s]


Processing BM25 for queries in zh...


Retrieving for zh: 100%|██████████| 76/76 [00:00<00:00, 776.66it/s]


In [8]:
bm25_results_df = pd.DataFrame([
    {"query_id": query_id, "query_lang": query_lang, "doc_id": doc_id, "score": score}
    for (query_id, query_lang), docs in bm25_results.items()
    for doc_id, score in docs
])

In [9]:
print(bm25_results_df.groupby('query_lang')['query_id'].nunique())
bm25_results_df.head()

query_lang
fa    76
ru    76
zh    76
Name: query_id, dtype: int64


,query_id,query_lang,doc_id,score
0,200,fa,de8bb4aa-a22f-4c47-9470-ca75a674b489,29.132540
1,200,fa,cc1cdb83-6e1b-4239-85b3-e30a19c90772,27.433842
2,200,fa,cf7ada1c-be4d-4293-bb8b-a69bb78f6edf,22.779131
3,200,fa,e48abf16-9e3d-4c04-9f6f-e2e3d8da4de7,21.519012
4,200,fa,4b148174-3bbf-4dd5-ae6e-9523b2c70e5e,21.434446


## Evaluation

In [ ]:
# filtered qrels dataset for evaluation 
valid_doc_ids = set(documents_df['doc_id'])
qrels_df = pd.DataFrame(dataset.qrels_iter())
qrels_df = qrels_df[qrels_df['doc_id'].isin(valid_doc_ids)]
print(qrels_df.shape[0])

15933


In [11]:
import ir_measures
from ir_measures import ScoredDoc, Qrel

qrels = [
    ir_measures.Qrel(query_id=row['query_id'], doc_id=row['doc_id'], relevance=row['relevance'])
    for _, row in qrels_df.iterrows()
]

results = [
    ScoredDoc(query_id=row['query_id'], doc_id=row['doc_id'], score=row['score'])
    for _, row in bm25_results_df.iterrows()
]

metrics = [
    ir_measures.nDCG @ 20,   # nDCG@20
    ir_measures.AP,          # Average Precision
    ir_measures.RBP(rel=1),
    ir_measures.R @ 100,     # Recall@100
    ir_measures.R @ 1000     # Recall@1000
]

# Evaluate
evaluation = ir_measures.calc_aggregate(metrics, qrels, results)

print("Evaluation Metrics:")
for metric, value in evaluation.items():
    print(f"{metric}: {value}")


Evaluation Metrics:
AP: 0.0848687071753339
R@100: 0.1646446392334767
nDCG@20: 0.21873961448806123
R@1000: 0.1646446392334767
RBP(rel=1): 0.13168194380350867
